# TRADES

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '0,1,2,3'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/trades/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.1
momentum = 0.9
weight_decay = 2e-4
batch_size = 128
total_epochs = 100
beta = 6.0
epsilon = 8/255
alpha = 2/255
num_repeats = 10

## Inner maximization

In [3]:
def inner_max(model, kl, inputs, targets, epsilon, alpha, num_repeats):
    noise = torch.FloatTensor(inputs.shape).normal_(0, 1).cuda()
    x = torch.clamp(inputs + 0.001*noise, min=0, max=1)
    
    model.eval()
    for _ in range(num_repeats):
        x.requires_grad_()
        with torch.enable_grad():
            loss = kl(torch.log_softmax(model(x), dim=1),
                      torch.softmax(model(inputs), dim=1))
        loss.backward()
        grads = x.grad.data
        x = x.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs-epsilon), inputs+epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def training(epoch, model, dataloader, optimizer, num_classes, beta=6.0, epsilon=8/255, alpha=2/255, num_repeats=10):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
        
    kl = nn.KLDivLoss(size_average=False)
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        x = inner_max(model, kl, inputs, targets, epsilon, alpha, num_repeats)
        model.train()
        logits_adv = model(x)
        logits_nat = model(inputs)
        
        xent_loss = xent(logits_nat, targets)
        kl_loss = kl(F.log_softmax(logits_adv, dim=1),F.softmax(logits_nat, dim=1)) / batch
        loss = xent_loss + beta*kl_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/len(dataloader),
                     num_correct/batch, total_correct/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
        
            noise = torch.FloatTensor(inputs.shape).uniform_(-epsilon,epsilon).cuda()
            x = torch.clamp(inputs+noise, min=0, max=1)
            for _ in range(num_repeats):
                x.requires_grad_()
                logits = model(x)
                loss = xent(logits, targets)
                loss.backward()
                grads = x.grad.data
                x = x.detach() + alpha*torch.sign(grads).detach()
                x = torch.min(torch.max(x, inputs - epsilon), inputs + epsilon).clamp(min=0, max=1)
        
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
        
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
        
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(),lr=lr, momentum=momentum, weight_decay=weight_decay)

scheduler = [75, 90]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes, beta, epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)
        
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch 0 [0/383] | loss: 2.3491 (avg: 0.0061) | acc: 0.0938 (avg: 0.0938) |
Epoch 0 [100/383] | loss: 1.9437 (avg: 0.5729) | acc: 0.2969 (avg: 0.2277) |
Epoch 0 [200/383] | loss: 2.0697 (avg: 1.0878) | acc: 0.2578 (avg: 0.2701) |
Epoch 0 [300/383] | loss: 1.9368 (avg: 1.5902) | acc: 0.3047 (avg: 0.2946) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:29<00:00,  4.78it/s, acc_nat=0.401, acc_rob=0.25]


Epoch 1 [0/383] | loss: 1.9651 (avg: 0.0051) | acc: 0.3438 (avg: 0.3438) |
Epoch 1 [100/383] | loss: 1.9583 (avg: 0.4820) | acc: 0.3516 (avg: 0.3932) |
Epoch 1 [200/383] | loss: 1.8554 (avg: 0.9496) | acc: 0.3906 (avg: 0.4037) |
Epoch 1 [300/383] | loss: 1.7049 (avg: 1.4103) | acc: 0.4219 (avg: 0.4136) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [03:40<00:00,  4.54it/s, acc_nat=0.864, acc_rob=0.552]


Epoch 98 [0/383] | loss: 0.4340 (avg: 0.0011) | acc: 0.9219 (avg: 0.9219) |
Epoch 98 [100/383] | loss: 0.5482 (avg: 0.1250) | acc: 0.8906 (avg: 0.9347) |
Epoch 98 [200/383] | loss: 0.4353 (avg: 0.2447) | acc: 0.9375 (avg: 0.9349) |
Epoch 98 [300/383] | loss: 0.6128 (avg: 0.3633) | acc: 0.9219 (avg: 0.9352) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:35<00:00,  4.63it/s, acc_nat=0.863, acc_rob=0.538]


Epoch 99 [0/383] | loss: 0.4306 (avg: 0.0011) | acc: 0.9453 (avg: 0.9453) |
Epoch 99 [100/383] | loss: 0.3438 (avg: 0.1226) | acc: 0.9609 (avg: 0.9373) |
Epoch 99 [200/383] | loss: 0.4252 (avg: 0.2429) | acc: 0.9375 (avg: 0.9367) |
Epoch 99 [300/383] | loss: 0.4694 (avg: 0.3604) | acc: 0.9297 (avg: 0.9385) |


[eval] 847/1000:  85%|████████▍ | 846/1000 [03:00<00:31,  4.94it/s, acc_nat=0.731, acc_rob=0.462]